In [ ]:
%load_ext autoreload
%autoreload 2

# Using *Py-Dreambooth* on a Local Jupyter Notebook 🧑‍💻
* Use *Py-Dreambooth* to easily create AI avatar images from photos of you, your family, friends, or pets!
* Please run on a machine with a GPU of 16GB or more.

## Install the package
* Install the *Py-Dreambooth* python package as shown below.

In [ ]:
!pip install -q py_dreambooth

## Import modules
* There are several types of model classes, but you'll be using the most basic model, the Stable Diffusion Dreambooth model `SDDreamboothModel`, but you don't need to worry about that right now. 🤷‍♂️

In [ ]:
from py_dreambooth.dataset import LocalDataset
from py_dreambooth.model import SdDreamboothModel
from py_dreambooth.predictor import LocalPredictor
from py_dreambooth.trainer import LocalTrainer
from py_dreambooth.utils.image_helpers import display_images
from py_dreambooth.utils.prompt_helpers import make_prompt

## Prepare your data 📸

In [ ]:
DATA_DIR = "data"  # the directory with photos for the model to train on 
OUTPUT_DIR = "models"  # The directory where the trained model files will be located

dataset = LocalDataset(DATA_DIR)

* Very important! In the `DATA_DIR` defined above, put the pictures (jpg or png) of the subject you want to train.
* For this task, you'll need about 10 to 20 solo, high-quality selfies taken with different backgrounds, lighting, and facial expressions. I think a great example can be found in [Joe Penna's GitHub repository](https://github.com/JoePenna/Dreambooth-Stable-Diffusion).

![Samples](../assets/asset-001.png)
* Use the following image processing method to crop the images into a square centered on the face. If the subject the model is trying to learn is not a person (for example, a dog), set the `detect_face` argument to `False`. 

In [ ]:
dataset = dataset.preprocess_images(detect_face=True)

## Train the model 🤖
* Now it's time to train the model! Tell the model the name of the subject you want to train (e.g., Joe) and the class it belongs to. 
* When defining a model, one of the important arguments is how many iterations to train, or `max_train_steps`. It is generally accepted that 800 to 1200 steps are appropriate for a person, and 200 to 400 steps are appropriate for a non-human animal. The default value is 75 times the number of photos you have. You don't need to worry about that right now 🤷‍♂️, but if you don't like the results of the generated image below, this is the first parameter to adjust.

In [ ]:
SUBJECT_NAME = "sks"  # The name of the subject you want to learn
CLASS_NAME = "person"  # The class to which the subject you want to learn belongs

model = SdDreamboothModel(
    subject_name=SUBJECT_NAME, 
    class_name=CLASS_NAME,
    # max_train_steps=1000,
)

trainer = LocalTrainer(output_dir=OUTPUT_DIR)

* Model training time can be as short as a few tens of minutes or as long as several hours.

In [ ]:
%%time
predictor = trainer.fit(model, dataset)

* If you restart the notebook kernel and then want to reload the models you've already trained, you can do so as follows.

In [ ]:
# predictor = LocalPredictor(model, OUTPUT_DIR)

## Create images as you wish! 💃
* Use the prompts to create any image you like. The prompt text should contain the subject name and class name defined above.
* Having trouble coming up with a good prompt? Don't worry. You can use the `make_prompt` function to generate a curated prompt at random. Check this out. 🙆‍♀️
* Creating great images takes patience. Play around with the prompts, but if the quality of the generation itself is problematic, you may need to retrain with better data and more appropriate training parameters.

In [ ]:
%%time
prompt = f"A photo of {SUBJECT_NAME} {CLASS_NAME} with Eiffel Tower in the background"
# prompt = next(make_prompt(SUBJECT_NAME, CLASS_NAME))

print(f"The prompt is as follows:\n{prompt}")

images = predictor.predict(
    prompt, height=768, width=512, num_images_per_prompt=2,
)

display_images(images, fig_size=10)